In [117]:
import plotly.graph_objects as go
from datetime import datetime
import pandas as pd

- **Load Data**

In [118]:
BPS_df = pd.read_csv('data/BPS.csv')

BPS_df['Period'] = pd.to_datetime(BPS_df['Period'], format="%b-%Y")

BPS_df.set_index('Period', inplace=True)

BPS_df['^%'] = BPS_df['Value'].pct_change()

In [119]:
fig = go.Figure()

y_start = [(-100 + BPS_df['Value'].min()) for _ in range(len(BPS_df))]

fig.add_trace(go.Scatter(
    x=BPS_df.index, 
    y=y_start, 
    name='-',
    line=dict(width=0),
    hoverinfo="skip",
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=BPS_df.index,
    y=BPS_df['Value'],
    name='BPS',
    fill='tonexty',
    fillcolor='rgba(0, 0, 255, 0.9)',
    line=dict(color='blue', width=1),
))

fig.add_trace(go.Scatter(
    x=BPS_df.index, 
    y=[BPS_df['Value'].mean() for _ in range(len(BPS_df))], 
    name='Mean',
    line=dict(color='red', width=1),
    showlegend=True
))

fig.update_layout(
    title=dict(text='Building Permits Survey (New residential construction)'),
    width=1400,
    height=500,
    plot_bgcolor="black",
    paper_bgcolor="#212121",
    font=dict(color="white"),
    hovermode="x unified",
    yaxis=dict(title="Thousands of units", exponentformat="none", showgrid=False),
    barmode="group",
    legend=dict(x=0.5, y=-0.2, orientation="h"),
    xaxis=dict(showgrid=False),
)

fig.show()

- **GDP data**

In [120]:
GDP_df = pd.read_csv('data/A191RL1Q225SBEA.csv')

GDP_df['DATE'] = pd.to_datetime(GDP_df['DATE'])
GDP_df.set_index('DATE', inplace=True)
GDP_df.columns = ['GDP']

In [121]:
fig = go.Figure()

filtered_gdp_df = GDP_df[GDP_df.index >= BPS_df.index[0]]

fig.add_trace(go.Bar(
    x=filtered_gdp_df.index,
    y=filtered_gdp_df['GDP'],
    name='GDP',
    marker=dict(color='yellow'),
    yaxis="y2"  
))

fig.add_trace(go.Scatter(
    x=BPS_df.index,
    y=BPS_df["Value"],
    name='BPS',
    # fill='tonexty',
    # fillcolor='rgba(0, 0, 255, 0.9)',
    line=dict(color='white', width=1),
))

# ------------------------------------------------------------------------
level_a = 1100  # Level on left y-axis
level_b = 0  # Matching level on right y-axis

# Define the ranges for each axis
range_left = [BPS_df["Value"].min() - 100, BPS_df["Value"].max() + 100]  # Min and max for right y-axis (to be adjusted)
range_right = [filtered_gdp_df['GDP'].min() - 5, filtered_gdp_df['GDP'].max() + 5]  # Min and max for left y-axis

# Calculate scaling factor to align levels
proportion = (level_a - range_left[0]) / (range_left[1] - range_left[0])  # Position of level_a
range_right[1] = range_right[0] + (level_b - range_right[0]) / proportion  # Adjust right range
# ------------------------------------------------------------------------

fig.update_layout(
    title=dict(text='Building Permits Authorised vs GDP change'),
    width=1400,
    height=900,
    plot_bgcolor="black",
    paper_bgcolor="#212121",
    font=dict(color="white"),
    hovermode="x unified",
    yaxis=dict(title="Building Permits Authorised (Thousands of units)", exponentformat="none", range=range_left, showgrid=False),
    yaxis2=dict(title="Real GDP change in %", overlaying="y", side="right", range=range_right, showgrid=False, zeroline=False),
    barmode="group",
    legend=dict(x=0.5, y=-0.2, orientation="h"),
    xaxis=dict(showgrid=False),
)

fig.show()

- **BPS pct change**

In [122]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=BPS_df.index,
    y=BPS_df['^%'] * 100,
    name='BPS pct change',
    marker=dict(color='blue'),
))

fig.add_trace(go.Scatter(
    x=BPS_df.index, 
    y=[100 * BPS_df['^%'].mean() for _ in range(len(BPS_df))], 
    name='Mean',
    line=dict(color='red', width=1),
    showlegend=True
))

fig.update_layout(
    title=dict(text='Building Permits Authorised pct change %'),
    width=1400,
    height=900,
    plot_bgcolor="black",
    paper_bgcolor="#212121",
    font=dict(color="white"),
    hovermode="x unified",
    yaxis=dict(title="%", exponentformat="none", showgrid=False, zeroline=False),
    barmode="group",
    legend=dict(x=0.5, y=-0.2, orientation="h"),
    xaxis=dict(showgrid=False),
)

fig.show()